Alegem un llm mic pentru a testa capacitatea acestuia de a genera cereri

In [ ]:
#!pip install transformers datasets peft accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import torch
model_name = "dumitrescustefan/gpt-neo-romanian-780M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

Criterii pe care trebuie sa le avem in considerare:
*   parti implicate
*   contextul/scopul
*   tipul actiunii

Pentru fiecare dintre acestea alegem diferite valori pe care sa la ia aceste criterii in prompt-ul pe care il vom trimite llm-ului.









In [ ]:
# 1. Părțile implicate
parti_implicate = [
    "persoană fizică",
    "persoană juridică",
    "instituție publică",
    "organizație non-guvernamentală",
    "societate comercială",
    "autoritate locală",
    "asociație de proprietari",
    "profesionist independent (PFA, II, IF)",
    "statul român",
    "minister"
]

# 2. Contextul / scopul
context_scop = [
    "reglementare",
    "dovadă",
    "soluționare a unui conflict juridic",
    "constatare a unui fapt juridic",
    "protecția unui drept subiectiv",
    "obținerea unei despăgubiri",
    "anularea unui act administrativ",
    "executarea unei obligații",
    "recuperarea unei creanțe",
    "stabilirea filiației"
]

# 3. Tipul cererii / acțiunii juridice
tip_actiune = [
    "cerere de chemare în judecată",
    "plângere contravențională",
    "contestație la executare",
    "cerere de ordonanță președințială",
    "cerere de divorț",
    "acțiune în revendicare",
    "cerere de anulare act administrativ",
    "acțiune în constatare",
    "plângere penală",
    "apel / recurs"
]




Promt-ul pe care il vom trimite va avea aceasta forma:
```
prompt = (
    f"Creează o {actiune} fictivă pe baza următoarelor elemente:\n"
    f"- Reclamant: un {parte_reclamant} responsabil\n"
    f"- Pârât: un {parte_parat} neglijent\n"
    f"- Context / scop: {scop}\n"
    f"Documentul trebuie să fie clar, concis și formal, maxim 100 de cuvinte, include o sectiunde pentru data, una pentru semnatura si una pentru stampila"
)
```



In [ ]:
import torch
import random
parte_reclamant = random.choice(parti_implicate)
parte_parat = random.choice(parti_implicate)
scop = random.choice(context_scop)
actiune = random.choice(tip_actiune)

device = "cuda" if torch.cuda.is_available() else "cpu"
prompt = (
    f"Creează o {actiune} fictivă pe baza următoarelor elemente:\n"
    f"- Reclamant: un {parte_reclamant} responsabil\n"
    f"- Pârât: un {parte_parat} neglijent\n"
    f"- Context / scop: {scop}\n"
    f"Documentul trebuie să fie clar, concis și formal, maxim 100 de cuvinte, include o sectiunde pentru data, una pentru semnatura si una pentru stampila"
)

inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_length=250,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Creează o contestație la executare fictivă pe baza următoarelor elemente:
- Reclamant: un persoană juridică responsabil
- Pârât: un statul român neglijent
- Context / scop: protecția unui drept subiectiv
Documentul trebuie să fie clar, concis și formal, maxim 100 de cuvinte, include o sectiunde pentru data, una pentru semnatura si una pentru stampila.
- Formular de plângere (în limba română)
- Formular de cerere (în limba română)
- Cerinte:
- Expunere: un rezumat al faptelor și circumstanțelor
- Cerere de executare silită (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de anulare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a execu

Incercam antrenarea cu un set mic de data, folosim LoRA pentru a rula mai repede

In [ ]:
from peft import LoraConfig
from peft import get_peft_model

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_proj","c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

import json


with open("cereri.json", "r", encoding="utf-8") as f:
    data = json.load(f)


print(f"S-au încărcat {len(data)} cereri de chemare în judecată.")


print(data[0]["text"])

dataset = Dataset.from_list(data)



S-au încărcat 123 cereri de chemare în judecată.
### CERERE DE CHEMARE ÎN JUDECATĂ
Reclamant: Ion Popescu
Pârât: SC Exemplu SRL
Obiect: Recuperare sumă de 10.000 lei
Introducere: Subsemnatul Ion Popescu formulez prezenta cerere pentru recuperarea sumei datorate de SC Exemplu SRL.
Motivare: Pârâtul nu a achitat factura nr. 123/2025 în termenul legal.
Solicitări: Solicit instanței obligarea pârâtului la plata sumei de 10.000 lei, precum și a dobânzilor și a cheltuielilor de judecată.


In [ ]:
def tokenize_and_add_labels(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(tokenize_and_add_labels, batched=True)


Map:   0%|          | 0/123 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt-neo-ro-juridic",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    learning_rate=5e-5,
    fp16=True if torch.cuda.is_available() else False,
    report_to=[],
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
10,13.391000
20,12.382500
30,9.565400
40,5.292900
50,1.590400
60,0.604700
70,0.591300
80,0.596100
90,0.603700
100,0.577200


TrainOutput(global_step=123, training_loss=3.78235139885569, metrics={'train_runtime': 36.2038, 'train_samples_per_second': 3.397, 'train_steps_per_second': 3.397, 'total_flos': 257442179973120.0, 'train_loss': 3.78235139885569, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./gpt-neo-ro-juridic")
tokenizer.save_pretrained("./gpt-neo-ro-juridic")

('./gpt-neo-ro-juridic/tokenizer_config.json',
 './gpt-neo-ro-juridic/special_tokens_map.json',
 './gpt-neo-ro-juridic/vocab.json',
 './gpt-neo-ro-juridic/merges.txt',
 './gpt-neo-ro-juridic/added_tokens.json',
 './gpt-neo-ro-juridic/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Folosim device-ul: {device}")


base_model_name = "dumitrescustefan/gpt-neo-romanian-780M"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None
)


lora_model_path = "./gpt-neo-ro-juridic"
model = PeftModel.from_pretrained(base_model, lora_model_path)
model.eval()
model.to(device)


system_message = (
    "Ești un asistent juridic expert, care generează documente legale clare, coerente și structurate. "
    "Urmează limbajul juridic român, formal, concis. "
    "Documentele trebuie să fie structurate astfel: titlu, părți, introducere, expunere de fapt, obligații, fundament juridic, concluzie. "
    "Include: temei contractual, obligații, clauze, penalități, daune, onorarii avocați, cheltuieli, prejudicii."
)

parte_reclamant = random.choice(parti_implicate)
parte_parat = random.choice(parti_implicate)
scop = random.choice(context_scop)
actiune = random.choice(tip_actiune)

device = "cuda" if torch.cuda.is_available() else "cpu"
prompt_user = (
    f"Creează o {actiune} fictivă pe baza următoarelor elemente:\n"
    f"- Reclamant: un {parte_reclamant} responsabil\n"
    f"- Pârât: un {parte_parat} neglijent\n"
    f"- Context / scop: {scop}\n"
    f"Documentul trebuie să fie clar, concis și formal, maxim 100 de cuvinte, include o sectiunde pentru data, una pentru semnatura si una pentru stampila"
)

prompt = system_message + "\n\n" + prompt_user


inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=350,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=50,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )


print("\n=== Document generat ===\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Folosim device-ul: cuda

=== Document generat ===

Ești un asistent juridic expert, care generează documente legale clare, coerente și structurate. Urmează limbajul juridic român, formal, concis. Documentele trebuie să fie structurate astfel: titlu, părți, introducere, expunere de fapt, obligații, fundament juridic, concluzie. Include: temei contractual, obligații, clauze, penalități, daune, onorarii avocați, cheltuieli, prejudicii.

Creează o cerere de ordonanță președințială fictivă pe baza următoarelor elemente:
- Reclamant: un persoană juridică responsabil
- Pârât: un persoană fizică neglijent
- Context / scop: protecția unui drept subiectiv
Documentul trebuie să fie clar, concis și formal, maxim 100 de cuvinte, include o sectiunde pentru data, una pentru semnatura si una pentru stampila.
Pentru a începe procedura de suspendare a executării silite, trebuie să depui o cerere la instanța competentă. Cererea trebuie să conțină următoarele elemente:
- Titlul executoriu
- Durata de exec

In [ ]:
expected=["""CERERE DE ORDONANȚĂ PREȘEDINȚIALĂ

Subsemnata S.C. Protecții S.R.L., în calitate de reclamant, solicit instanței emiterea unei ordonanțe președințiale împotriva domnului Ion Ionescu, în calitate de pârât, pentru protecția dreptului nostru subiectiv de proprietate intelectuală.

Se solicită ca pârâtul să fie obligat să înceteze imediat orice utilizare neautorizată a materialelor protejate până la soluționarea definitivă a litigiului, pentru prevenirea unui prejudiciu ireparabil.

Anexez documentele justificative.

Data: …………………
Semnătura reclamantului: …………………
Ștampila societății: …………………""",
          """ACȚIUNE ÎN REVENDICARE

Reclamant: Autoritate Locală Responsabil
Pârât: Persoană Juridică Neglijent

Subsemnata Autoritate Locală Responsabil formulează prezenta acțiune în revendicare împotriva Persoanei Juridice Neglijente, solicitând recunoașterea dreptului de proprietate și recuperarea bunului în litigiu. Scopul acțiunii îl constituie obținerea dovezii deținute de pârât și restabilirea drepturilor legale ale reclamantului. Cererea se întemeiază pe neglijența pârâtului în respectarea obligațiilor sale.

Data: …………………

Semnătura reclamantului: …………………

Ștampilă: …………………""",
          """Contestație la executare

Reclamant: Persoană juridică responsabilă
Pârât: Statul Român

Obiect: Contestație la executare

Expunere: Subsemnata persoană juridică formulează prezenta contestație împotriva măsurilor de executare aplicate de pârât, având ca scop protecția dreptului subiectiv garantat de lege. Solicităm suspendarea procedurii de executare și reevaluarea conform prevederilor legale, în vederea respectării drepturilor noastre legitime.

Data: _______________________
Semnătura reprezentantului reclamantului: _______________________
Ștampilă: _______________________"""

          ]
response=["""Pentru a începe procedura de suspendare a executării silite, trebuie să depui o cerere la instanța competentă. Cererea trebuie să conțină următoarele elemente:
- Titlul executoriu
- Durata de executare
- Data de emitere a somației de plată
- Durata de executare
- Data scadenței
- Suma datorată
- Suma minimă de plată
În cerere trebuie să enumeri motivele de fapt și de drept pentru care vrei să ceri suspendarea executării silite. Dacă e necesar, poți solicita și o cauțiune.
Trebuie să prezinți situația ta juridică. Este posibil să fii obligat la plata unor daune interese dacă cererea ta de suspendare este respinsă. De asemenea, trebuie să prezinți situația financiară a debitorului.
Depune cererea la instanța competentă.
Dacă cererea ta este admisă de instanță, ți se va acorda un termen de grație de o lună pentru a-ți plăti datoria.
Dacă nu plătești datoria, instanța va pronunța o hotărâre prin care îți va solicita să restitui bunurile sau banii. Instanța va dispune în acest caz suspendarea executării silite.
Dacă instanța constată că nu ai suficiente fonduri pentru a achita datoria, poate să-ți impună o amendă civilă.
- Durata procedurii: 30 de zile
Dacă sentința nu se execută, se va pronunța o hotărâre de suspendare a executării silite. Dacă sentința nu se execută, se va pronunța o hotărâre de suspendare a executării silite.
Dacă sentința nu se execută, se va pronunța o hotărâre de suspendare a executării silite. Dacă sentința nu se execută, se va pronunța o hotărâre de suspendare a executării silite.
În cazul în care sentința nu se execută, instanța va solicita executorului judecătoresc să o pună în executare. Executorul judecătoresc va executa sentința. Instanța va dispune suspendarea executării silite. Executorul judecătoresc va suspenda executarea silită.""",
"""Documentul trebuie să fie însoțit de toate documentele justificative.
Documentul trebuie să fie emis în limba română, engleză sau franceză, iar traducerea trebuie să fie legalizată.
În cazul în care nu este emis în limba română, documentul trebuie să fie semnat de un notar public sau de un traducător autorizat.
În cazul în care nu este emis în limba română, documentul trebuie să fie semnat de un notar public sau de un traducător autorizat.
În cazul în care documentul nu este emis în limba română, documentul trebuie să fie semnat de un notar public sau de un traducător autorizat.
În cazul în care documentul nu este emis în limba română, documentul trebuie să fie semnat de un notar public sau de un traducător autorizat.
Documentul trebuie să fie însoțit de toate documentele justificative.
Documentele justificative trebuie să fie în format electronic, în format PDF, astfel:
- copia documentului original;
- o""",
          """- Formular de plângere (în limba română)
- Formular de cerere (în limba română)
- Cerinte:
- Expunere: un rezumat al faptelor și circumstanțelor
- Cerere de executare silită (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de anulare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română)
- Cerere de suspendare a executării silite (în limba română"""
          ]

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
i=0
final_score=0
for exp in expected:
  bleu_score = sentence_bleu([exp], response[i])
  final_score+=bleu_score
  i+=1
print("BLEU score:", final_score/i)


BLEU score: 0.1733408139674


BLUE score-ul obtinut este foarte mic: 0,17

In [ ]:

prompt_user_v2 = ("Completeaza aceasta cerere"
   """### CERERE DE CHEMARE ÎN JUDECATĂ
Reclamant:"""
)
inputs = tokenizer(prompt_user_v2, return_tensors="pt", padding=True, truncation=True)


inputs = {k: v.to("cpu") for k, v in inputs.items()}


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=350,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=50,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )


print("\n=== Document generat ===\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


=== Document generat ===

Completeaza aceasta cerere### CERERE DE CHEMARE ÎN JUDECATĂ
Reclamant: ## ALTE VÂNZĂTORI
Subsemnatul: ## REVENIREA ÎN FUNCȚIE A LUI JULIET.
Stimate domn, Vă rog să-mi permiteți a vă aduce la cunoștință că am primit cu mare plăcere scrisoarea dumneavoastră, în care îmi faceți cunoscute rezultatele cercetărilor dumneavoastră asupra morții lui Juliette Mauclin.
Am fost foarte interesat de această problemă. Vă voi scrie în continuare, cu toată sinceritatea, despre acest lucru.
În dimineața zilei de 28 septembrie, m-am dus în satul Montserrat și am găsit cadavrul lui Juliette Mauclin întins pe o suprafață de aproximativ 3 metri pătrați. Era îmbrăcat cu o rochie de mătase de culoare deschisă și avea o expresie de bucurie pe față.
Din păcate, din cauza unei răni de la cap, nu puteam vedea nici o rană în partea superioară a feței și în partea de sus a capului.
Mi-am dat seama că a fost lovită.
Am luat o foarfecă și am tăiat-o de la cap la picioare.
Cu foarfecele, am 

Incercam sa utilizam un alt llm

In [ ]:
!pip install transformers torch

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Folosește dispozitivul:", device)

model_name = "iliemihai/gpt-neo-romanian-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

parte_reclamant = random.choice(parti_implicate)
parte_parat = random.choice(parti_implicate)
scop = random.choice(context_scop)
actiune = random.choice(tip_actiune)

device = "cuda" if torch.cuda.is_available() else "cpu"
prompt = (
    f"Creează o {actiune} fictivă pe baza următoarelor elemente:\n"
    f"- Reclamant: un {parte_reclamant} responsabil\n"
    f"- Pârât: un {parte_parat} neglijent\n"
    f"- Context / scop: {scop}\n"
    f"Documentul trebuie să fie clar, concis și formal, maxim 100 de cuvinte, include o sectiunde pentru data, una pentru semnatura si una pentru stampila"
)


inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_length=250,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Folosește dispozitivul: cuda


tokenizer_config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/593M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/593M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Creează o plângere penală fictivă pe baza următoarelor elemente:
- Reclamant: un organizație non-guvernamentală responsabil
- Pârât: un instituție publică neglijent
- Context / scop: dovadă
Documentul trebuie să fie clar, concis și formal, maxim 100 de cuvinte, include o sectiunde pentru data, una pentru semnatura si una pentru stampila. „De aceea, în cazul în care vă decideți să depuneți plângerea în instanță, veți primi o copie a plângerii. Pentru a înțelege mai bine modul în care funcționează acest proiect, este de asemenea util să faceți o scurtă descriere a modului în care puteți face acest lucru. Aceasta reclama ne permite sa iti oferim Caruselul cu Cute Batman gratuit in 15 secunde. Nu știu de ce, dar am început să citesc cărți din toate domeniile. Acest lucru este de ce este important să se ia doar pură și produse de calitate in Mehedinti România. A se vedea, de asemenea,Prognoza meteo în Regatul Unit țară, prognoza meteo pe oră în Shrewsbury, vremea în Shrewsbury astăzi, vreme

AM incercat poezii pentru a vedea daca merge mai bine

In [ ]:
prompt = (
    "Pasari zboare prin vazduh"

)

inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_length=250,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9,
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Pasari zboare prin vazduh de vant. S-ar parea ca te afli pe o pagina a unui ziar. Ca sa afli ca nu mai e vorba de mine. Si trebuie sa-ti fac fata pentru ca sunt femeie. Dar trebuie sa ma crezi pe cuvant. "Pe drum, voi pleca din tara, va voi lua ca sotie pe cea mai mare si singura pe care o am". Nu-mi vine sa cred ca cineva care a scris despre aceasta carte ar putea sa-si piarda viata. "si atunci, dupa ce vei primi scrisoarea, ma vei urma in India, si cand te voi vedea nu vei mai fi un indian care sa-mi poarte povara." Am plecat cu trenul din India, pe drumul din fata ta. Am crezut ca pot sa zbor din India. "si atunci, dupa ce vei primi scrisoarea, ma vei urma in India, si cand te voi vedea nu vei mai fi un indian care sa-mi poarte povara. Mi-a spus ca si-a ales deja destinul ca sa-si traiasca viata in India. Si ca el a fost omul care a pus in miscare destinul meu in India. Si-a dorit sa ma faca indian


Incercam varianta RAG - folosim primul model, 780M

In [ ]:
!pip install sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 28.1 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


texts = [item["text"] for item in data]


embedder = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = embedder.encode(texts)


index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))


In [ ]:
def retrieve(query, k=2):
    query_vec = embedder.encode([query])
    D, I = index.search(query_vec, k)
    return [texts[i] for i in I[0]]


In [ ]:
def rag_answer(query, k=2):
    context = "\n".join(retrieve(query, k))
    prompt = f"Context juridic:\n{context}\n\nÎntrebare: {query}\nRăspuns:"

    # Mutăm totul pe același device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=250, do_sample=True, temperature=0.7)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

query = """Creează o cerere de chemare în judecată pentru un proprietar care vrea să recupereze 8.000 lei de la o firmă care nu a plătit serviciile prestate."""
print(rag_answer(query))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Context juridic:
### CERERE DE CHEMARE ÎN JUDECATĂ
Reclamant: Andrei Popa
Pârât: SC Servicii SRL
Obiect: Recuperare chirie restantă 3.000 lei
Introducere: Subsemnatul Andrei Popa formulez prezenta cerere pentru recuperarea chiriei restante.
Motivare: Pârâtul nu a achitat chiria aferentă lunilor ianuarie–martie 2025.
Solicitări: Solicit instanței obligarea pârâtului la plata sumei de 3.000 lei și a cheltuielilor de judecată.
### CERERE DE CHEMARE ÎN JUDECATĂ
Reclamant: Ion Popescu
Pârât: SC Exemplu SRL
Obiect: Recuperare sumă de 10.000 lei
Introducere: Subsemnatul Ion Popescu formulez prezenta cerere pentru recuperarea sumei datorate de SC Exemplu SRL.
Motivare: Pârâtul nu a achitat factura nr. 123/2025 în termenul legal.
Solicitări: Solicit instanței obligarea pârâtului la plata sumei de 10.000 lei, precum și a dobânzilor și a cheltuielilor de judecată.

Întrebare: Creează o cerere de chemare în judecată pentru un proprietar care vrea să recupereze 8.000 lei de la o firmă care nu a plă

Concluzii:

*   Raspunsurile generate de ambele llm-uri nu sunt coerente si nu urmeaza instructiunile primite in promt
*   Ambele llm-uri incearca sa completeze promt-ul, nu incearca sa rapunda
*   Llm-ul mai mare da respunsuri mai structurate
*   Finetuning-ul nu a facut o diferenta majora in ceea ce priveste calitatea raspunsului



